### Improving Semantic Water Segmentation by Fusing Sentinel-1 Intensity and Interferometric Synthetic Aperture Radar (InSAR) Coherence Data

**Author: Ernesto Colon**

**The Cooper Union for the Advancement of Science and Art**

#### Download Sentinel-2 GRD products from Google Earth Engine

The workflow outlined in this notebook is used for generating Sentinel-1 chips for offline model training.

The bounding boxes used to generate the Sentinel-1 chips are extracted from the Sen1Floods11 data set.

QGIS is used to split the large geojson bounding box into a grid.

In [ ]:
# Import libraries
import geemap
import json
import ee
import os
from tqdm import tqdm
from datetime import datetime, timedelta

**Initialize Google Earth Engine**

In [ ]:
ee.Initialize()

### Wrapping the workflow into functions

The next workflow assumes that we already have geojson bounding boxes for the scenes of interest. These will be needed
to intersect the Sentinel-1 scenes in GEE.

Moreover, we have also split the bounding boxes from the Sen1Floods11 data set into a rectangular grid using QGIS. This
ensures that the requested file size from GEE is of a reasonable size.

In [ ]:
# define regions of interest
regions = ['Bolivia', 'USA', 'Mekong', 'Colombia', 'India', 'Paraguay', 'Sri-Lanka']
geojson_dir = "pth_to_geojson_dir"

# create dictionary with regional geojson paths
bbx_json_dict = {'Bolivia' : f"{geojson_dir}\\Bolivia.geojson",
                 'USA' : f"{geojson_dir}\\USA.geojson",
                 'Mekong' : f"{geojson_dir}\\Cambodia.geojson",
                 'Colombia' : f"{geojson_dir}\\Colombia.geojson",
                 'India' : f"{geojson_dir}\\India.geojson",
                 'Paraguay' : f"{geojson_dir}\\Paraguay.geojson",
                 'Sri-Lanka' : f"{geojson_dir}\\Sri-Lanka.geojson"}

# create dictionary with regional grid geojson paths
grid_dir = "pth_to_grid_dir"
grid_json_dict = {'Bolivia' : f"{grid_dir}\\Bolivia\\bolivia_grid.geojson",
                  'USA' : f"{grid_dir}\\USA\\usa_grid.geojson",
                 'Mekong' : f"{grid_dir}\\Mekong\\mekong_grid.geojson",
                 'Colombia' : f"{grid_dir}\\Colombia\\colombia_grid.geojson",
                 'India' : f"{grid_dir}\\India\\india_grid.geojson",
                 'Paraguay' : f"{grid_dir}\\Paraguay\\paraguay_grid.geojson",
                 'Sri-Lanka' : f"{grid_dir}\\Sri-Lanka\\srilanka_grid.geojson"}

Loop through the regions and extract the following:

- Region geometry

- Event dates for filtering

In [ ]:
bbx_geo_by_reg = dict()
s1_event_date_by_reg = dict()

grid_feat_by_reg = dict()
grid_geom_by_reg = {region : [] for region in regions}

for region in regions:

    with open(bbx_json_dict[region], 'r') as f:
        fc = json.load(f)

        # grab the WGS geometry
        bbx_geo_by_reg[region] = ee.Geometry(fc['features'][0]['geometry'])

        # grab the Sentinel-2 scene date
        s1_event_date_by_reg[region] = fc['features'][0]['properties']['s1_date'].replace('/', '-')

    with open(grid_json_dict[region]) as file:
        file_contents = json.load(file)

        grid_feat_by_reg[region] = file_contents['features']

        for rectangle in tqdm(range(len(grid_feat_by_reg[region]))):
            geom = grid_feat_by_reg[region][rectangle]['geometry']

            grid_geom_by_reg[region].append(ee.Geometry(geom))

Generate the event start and end dates by region

In [ ]:
days_to_subtract = 1
days_to_add = 1

event_dates = {}

for region in regions:
    event_date = s1_event_date_by_reg[region]

    start_date = str((datetime.fromisoformat(event_date) - timedelta(days=days_to_subtract)).date())
    end_date = str((datetime.fromisoformat(event_date) + timedelta(days=days_to_add)).date())

    event_dates[region] = [start_date, end_date]

### Generate the Sentinel-1 Mosaics by region

In [ ]:
def generate_s1_mosaic(start_date, end_date, aoi):
    """
    Function to generate Sentinel-1 mosaic using GEE
    :param start_date: string with valid date format ('YYYY-MM-DD')
    :param end_date: string with valid date format ('YYYY-MM-DD')
    :param aoi: GEE geometry with area of interest
    :return: GEE image with Sentinel-1 mosaic
    """
    # grab the image collection
    s1_ic = ee.ImageCollection('COPERNICUS/S1_GRD')

    # filter the image collection based on AOI and dates
    s1_ic = s1_ic.filterDate(start_date, end_date).filterBounds(aoi)

    #s1_ic.size().getInfo()

    # create mosaic
    s1_mosaic = s1_ic.mosaic()
    print(s1_mosaic.getInfo())

    return s1_mosaic

In [ ]:
mosaic_dict = {}

for region in regions:
    print(f"Generating Mosaic for region: {region}...\n")
    mosaic_dict[region] = generate_s1_mosaic(event_dates[region][0], event_dates[region][-1], bbx_geo_by_reg[region])

### Download the chips (based off the grids)

In [ ]:
import time
start_time = start_time = time.time()

out_dir = "out_dir_pth"

for region in regions:
    print(f"Currently downloading region: {region}.. \n\n")

    # create output directory if it does not already exist
    curr_out_dir = f"{out_dir}\\{region}"

    if not os.path.exists(curr_out_dir):
        os.mkdir(curr_out_dir)

    for idx, geom in tqdm(enumerate(grid_geom_by_reg[region])):
        # cut the mosaic to the aoi
        cut_chp = mosaic_dict[region].clip(geom).select(['VV', 'VH'])

        fname = f"{region}_s1_{idx}.tif"

        # save chip to disk
        geemap.ee_export_image(cut_chp, filename=f"{curr_out_dir}\\{fname}", scale=10, region=geom, file_per_band=False)

    print(f"Finished downloading region: {region}.. \n\n")

print(f"Total download time: {time.time() - start_time} seconds")

